<a href="https://colab.research.google.com/github/vektor8891/llm/blob/main/projects/12_bert_preparing/12_bert_preparing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Loading and Text Processing for BERT

## Tokenization and vocabulary building

### Tokenization

Components:

- `tokenizer`: tokenizes text
- `yield_tokens`: generator function that iterates through the data
- `word_dict`: defines special tokens used in text processing, such as padding `[PAD]`, class `[CLS]`, separator `[SEP]`, mask `[MASK]`, and unknown `[UNK]`
- `text_to_index`: converts text into a list of numerical indices based on the vocabulary
- `index_to_en`: translates sequence of indices back into readable English text.

Special tokens:

- **`CLS` (Classification Token)**: Start of Sentence (SOS) marker
- **`SEP` (Separator Token)**: End of Sentence (EOS) marker
- **`PAD` (Padding Token)**: added to sequences to ensure all inputs are of equal length
- **`MASK` (Masked Token)**: Utilized masked language modeling
- **`UNK` (Unknown Token)**: placeholder for unknown words

In [1]:
# !pip install torchtext==0.17.2
# !pip install portalocker==2.8.2
# !pip install transformers==4.35.2
# !pip install torch==2.2.0

In [2]:
from torchtext.data.utils import get_tokenizer

tokenizer = get_tokenizer("basic_english")

def yield_tokens(data_iter):
    for label, data_sample in data_iter:
        yield tokenizer(data_sample)

# Define special symbols and indices
PAD_IDX,CLS_IDX, SEP_IDX,  MASK_IDX,UNK_IDX= 0, 1, 2, 3, 4

# Make sure the tokens are in order of their indices to properly insert them in vocab
special_symbols = ['[PAD]','[CLS]', '[SEP]','[MASK]','[UNK]']

### Vocabulary building

In [5]:
from torchtext.datasets import IMDB
from itertools import chain
from itertools import islice

#create data splits
train_iter, test_iter = IMDB(split=('train', 'test'))
all_data_iter = chain(train_iter, test_iter)
#check tokenizer
# list(yield_tokens(all_data_iter))[5][:20]
fifth_item_tokens = next(islice(yield_tokens(all_data_iter), 5, None))
print(fifth_item_tokens[:20])

['i', 'would', 'put', 'this', 'at', 'the', 'top', 'of', 'my', 'list', 'of', 'films', 'in', 'the', 'category', 'of', 'unwatchable', 'trash', '!', 'there']


In [6]:
from torchtext.vocab import build_vocab_from_iterator

#create vocab : vocab is only built using train data
vocab=build_vocab_from_iterator(yield_tokens(all_data_iter),specials=special_symbols,special_first=True)

vocab.set_default_index(UNK_IDX)
VOCAB_SIZE=len(vocab)
print(VOCAB_SIZE)

147150


In [8]:
text_to_index=lambda text: [vocab(token) for token in tokenizer(text)]
index_to_en = lambda seq_en: " ".join([vocab.get_itos()[index] for index in seq_en])

seq_en = [0, 1, 2, 3, 4, 5, 6]  # Example input sequence
english_sentence = index_to_en(seq_en)
seq2=[6,16,26131]
english_sentence = index_to_en(seq2)

print(english_sentence)

text = "I've seen R-rated films with male nudity. Nowhere, because they don't exist."  # Example input text
text_to_index = lambda text: [vocab[token] for token in tokenizer(text)]
index_sequence = text_to_index(text)

print(index_sequence)

. i speculative
[16, 12, 149, 119, 11363, 117, 22, 928, 1047, 6, 1251, 7, 96, 42, 99, 12, 30, 1877, 6]


## Text masking and data preparation for BERT

### Text masking

In [9]:
import torch

def bernoulli_true_false(p):
    # Create a Bernoulli distribution with probability p
    bernoulli_dist = torch.distributions.Bernoulli(torch.tensor([p]))
    # Sample from this distribution and convert 1 to True and 0 to False
    return bernoulli_dist.sample().item() == 1

def Masking(token):
    # Decide whether to mask this token (20% chance)
    mask = bernoulli_true_false(0.2)

    # If mask is False, immediately return with '[PAD]' label
    if not mask:
        return token, '[PAD]'

    # If mask is True, proceed with further operations
    # Randomly decide on an operation (50% chance each)
    random_opp = bernoulli_true_false(0.5)
    random_swich = bernoulli_true_false(0.5)

    # Case 1: If mask, random_opp, and random_swich are True
    if mask and random_opp and random_swich:
        # Replace the token with '[MASK]' and set label to a random token
        mask_label = index_to_en(torch.randint(0, VOCAB_SIZE, (1,)))
        token_ = '[MASK]'

    # Case 2: If mask and random_opp are True, but random_swich is False
    elif mask and random_opp and not random_swich:
        # Leave the token unchanged and set label to the same token
        token_ = token
        mask_label = token

    # Case 3: If mask is True, but random_opp is False
    else:
        # Replace the token with '[MASK]' and set label to the original token
        token_ = '[MASK]'
        mask_label = token

    return token_, mask_label

In [10]:
torch.manual_seed(100)
for l in range(10):
  token="apple"
  token_,label=Masking(token)
  if token==token_ and label=="[PAD]":
    print(token_,label,f"\t Actual token *{token}* is left unchanged")
  elif token_=="[MASK]" and label==token:
    print(token_,label,f"\t Actual token *{token}* is masked with '{token_}'")
  else:
    print(token_,label,f"\t Actual token *{token}* is replaced with random token #{label}#")

[MASK] apple 	 Actual token *apple* is masked with '[MASK]'
apple [PAD] 	 Actual token *apple* is left unchanged
apple [PAD] 	 Actual token *apple* is left unchanged
apple [PAD] 	 Actual token *apple* is left unchanged
apple [PAD] 	 Actual token *apple* is left unchanged
[MASK] whored 	 Actual token *apple* is replaced with random token #whored#
apple [PAD] 	 Actual token *apple* is left unchanged
apple [PAD] 	 Actual token *apple* is left unchanged
apple [PAD] 	 Actual token *apple* is left unchanged
apple [PAD] 	 Actual token *apple* is left unchanged


### Data preparation for MLM

In [11]:
def prepare_for_mlm(tokens, include_raw_tokens=False):
    """
    Prepares tokenized text for BERT's Masked Language Model (MLM) training.

    """
    bert_input = []  # List to store sentences processed for BERT's MLM
    bert_label = []  # List to store labels for each token (mask, random, or unchanged)
    raw_tokens_list = []  # List to store raw tokens if needed
    current_bert_input = []
    current_bert_label = []
    current_raw_tokens = []

    for token in tokens:
        # Apply BERT's MLM masking strategy to the token
        masked_token, mask_label = Masking(token)

        # Append the processed token and its label to the current sentence and label list
        current_bert_input.append(masked_token)
        current_bert_label.append(mask_label)

        # If raw tokens are to be included, append the original token to the current raw tokens list
        if include_raw_tokens:
            current_raw_tokens.append(token)

        # Check if the token is a sentence delimiter (., ?, !)
        if token in ['.', '?', '!']:
            # If current sentence has more than two tokens, consider it a valid sentence
            if len(current_bert_input) > 2:
                bert_input.append(current_bert_input)
                bert_label.append(current_bert_label)
                # If including raw tokens, add the current list of raw tokens to the raw tokens list
                if include_raw_tokens:
                    raw_tokens_list.append(current_raw_tokens)

                # Reset the lists for the next sentence
                current_bert_input = []
                current_bert_label = []
                current_raw_tokens = []
            else:
                # If the current sentence is too short, discard it and reset lists
                current_bert_input = []
                current_bert_label = []
                current_raw_tokens = []

    # Add any remaining tokens as a sentence if there are any
    if current_bert_input:
        bert_input.append(current_bert_input)
        bert_label.append(current_bert_label)
        if include_raw_tokens:
            raw_tokens_list.append(current_raw_tokens)

    # Return the prepared lists for BERT's MLM training
    return (bert_input, bert_label, raw_tokens_list) if include_raw_tokens else (bert_input, bert_label)

In [12]:
torch.manual_seed(100)
original_input="The sun sets behind the distant mountains."
tokens=tokenizer(original_input)
bert_input, bert_label= prepare_for_mlm(tokens, include_raw_tokens=False)
print("Without raw tokens: \t ","\n \t original_input is: \t ", original_input,"\n \t bert_input is: \t ", bert_input,"\n \t bert_label is: \t ", bert_label)
print("-"*200)
torch.manual_seed(100)
bert_input, bert_label, raw_tokens_list= prepare_for_mlm(tokens, include_raw_tokens=True)
print("With raw tokens: \t ","\n \t original_input is: \t ", original_input,"\n \t bert_input is: \t ", bert_input,"\n \t bert_label is: \t ", bert_label,"\n \t raw_tokens_list is: \t ", raw_tokens_list)

Without raw tokens: 	  
 	 original_input is: 	  The sun sets behind the distant mountains. 
 	 bert_input is: 	  [['[MASK]', 'sun', 'sets', 'behind', 'the', '[MASK]', 'mountains', '.']] 
 	 bert_label is: 	  [['the', '[PAD]', '[PAD]', '[PAD]', '[PAD]', 'whored', '[PAD]', '[PAD]']]
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
With raw tokens: 	  
 	 original_input is: 	  The sun sets behind the distant mountains. 
 	 bert_input is: 	  [['[MASK]', 'sun', 'sets', 'behind', 'the', '[MASK]', 'mountains', '.']] 
 	 bert_label is: 	  [['the', '[PAD]', '[PAD]', '[PAD]', '[PAD]', 'whored', '[PAD]', '[PAD]']] 
 	 raw_tokens_list is: 	  [['the', 'sun', 'sets', 'behind', 'the', 'distant', 'mountains', '.']]


### Data preparation for NSP

In [14]:
import random

def process_for_nsp(input_sentences, input_masked_labels):
    """
    Prepares data for Next Sentence Prediction (NSP) task in BERT training.

    Args:
    input_sentences (list): List of tokenized sentences.
    input_masked_labels (list): Corresponding list of masked labels for the sentences.

    Returns:
    bert_input (list): List of sentence pairs for BERT input.
    bert_label (list): List of masked labels for the sentence pairs.
    is_next (list): Binary label list where 1 indicates 'next sentence' and 0 indicates 'not next sentence'.
    """
    if len(input_sentences) < 2:
       raise ValueError("must have two same number of items.")


    # Verify that both input lists are of the same length and have a sufficient number of sentences
    if len(input_sentences) != len(input_masked_labels):
        raise ValueError("Both lists must have the same number of items.")

    bert_input = []
    bert_label = []
    is_next = []

    available_indices = list(range(len(input_sentences)))

    while len(available_indices) >= 2:
        if random.random() < 0.5:
            # Choose two consecutive sentences to simulate the 'next sentence' scenario
            index = random.choice(available_indices[:-1])  # Exclude the last index
            # append list and add  '[CLS]' and  '[SEP]' tokens
            bert_input.append([['[CLS]']+input_sentences[index]+ ['[SEP]'],input_sentences[index + 1]+ ['[SEP]']])
            bert_label.append([['[PAD]']+input_masked_labels[index]+['[PAD]'], input_masked_labels[index + 1]+ ['[PAD]']])
            is_next.append(1)  # Label 1 indicates these sentences are consecutive

            # Remove the used indices
            available_indices.remove(index)
            if index + 1 in available_indices:
                available_indices.remove(index + 1)
        else:
            # Choose two random distinct sentences to simulate the 'not next sentence' scenario
            indices = random.sample(available_indices, 2)
            bert_input.append([['[CLS]']+input_sentences[indices[0]]+['[SEP]'],input_sentences[indices[1]]+ ['[SEP]']])
            bert_label.append([['[PAD]']+input_masked_labels[indices[0]]+['[PAD]'], input_masked_labels[indices[1]]+['[PAD]']])
            is_next.append(0)  # Label 0 indicates these sentences are not consecutive

            # Remove the used indices
            available_indices.remove(indices[0])
            available_indices.remove(indices[1])



    return bert_input, bert_label, is_next

In [15]:
#flatten the tensor
flatten = lambda l: [item for sublist in l for item in sublist]
# Sample input sentences
input_sentences = [["i", "love", "apples"], ["she", "enjoys", "reading", "books"], ["he", "likes", "playing", "guitar"]]
# Create masked labels for the sentences
input_masked_labels=[]
for sentence in input_sentences:
  _, current_masked_label= prepare_for_mlm(sentence, include_raw_tokens=False)
  input_masked_labels.append(flatten(current_masked_label))
# Create NSP pairs and labels
random.seed(100)
bert_input, bert_label, is_next = process_for_nsp(input_sentences, input_masked_labels)

# Print the output
print("BERT Input:")
for pair in bert_input:
    print(pair)
print("BERT Label:")
for pair in bert_label:
    print(pair)
print("Is Next: ", is_next)
print("-"*200)
random.seed(1000)
bert_input, bert_label, is_next = process_for_nsp(input_sentences, input_masked_labels)

# Print the output
print("BERT Input:")
for pair in bert_input:
    print(pair)
print("BERT Label:")
for pair in bert_label:
    print(pair)
print("Is Next: ", is_next)


BERT Input:
[['[CLS]', 'she', 'enjoys', 'reading', 'books', '[SEP]'], ['he', 'likes', 'playing', 'guitar', '[SEP]']]
BERT Label:
[['[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]'], ['he', '[PAD]', '[PAD]', '[PAD]', '[PAD]']]
Is Next:  [1]
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
BERT Input:
[['[CLS]', 'he', 'likes', 'playing', 'guitar', '[SEP]'], ['i', 'love', 'apples', '[SEP]']]
BERT Label:
[['[PAD]', 'he', '[PAD]', '[PAD]', '[PAD]', '[PAD]'], ['[PAD]', '[PAD]', '[PAD]', '[PAD]']]
Is Next:  [0]


### Finalizing BERT inputs

In [17]:
from copy import deepcopy

def prepare_bert_final_inputs(bert_inputs, bert_labels, is_nexts,to_tenor=True):
    """
    Prepare the final input lists for BERT training.
    """
    def zero_pad_list_pair(pair_, pad='[PAD]'):
        pair=deepcopy(pair_)
        max_len = max(len(pair[0]), len(pair[1]))
        #append [PAD] to each sentence in the pair till the maximum length reaches
        pair[0].extend([pad] * (max_len - len(pair[0])))
        pair[1].extend([pad] * (max_len - len(pair[1])))
        return pair[0], pair[1]

    #flatten the tensor
    flatten = lambda l: [item for sublist in l for item in sublist]
    #transform tokens to vocab indices
    tokens_to_index=lambda tokens: [vocab[token] for token in tokens]

    bert_inputs_final, bert_labels_final, segment_labels_final, is_nexts_final = [], [], [], []

    for bert_input, bert_label,is_next in zip(bert_inputs, bert_labels,is_nexts):
        # Create segment labels for each pair of sentences
        segment_label = [[1] * len(bert_input[0]), [2] * len(bert_input[1])]

        # Zero-pad the bert_input and bert_label and segment_label
        bert_input_padded = zero_pad_list_pair(bert_input)
        bert_label_padded = zero_pad_list_pair(bert_label)
        segment_label_padded = zero_pad_list_pair(segment_label,pad=0)

        #convert to tensors
        if to_tenor:

            # Flatten the padded inputs and labels, transform tokens to their corresponding vocab indices, and convert them to tensors
            bert_inputs_final.append(torch.tensor(tokens_to_index(flatten(bert_input_padded)),dtype=torch.int64))
            #bert_labels_final.append(torch.tensor(tokens_to_index(flatten(bert_label_padded)),dtype=torch.int64))
            bert_labels_final.append(torch.tensor(tokens_to_index(flatten(bert_label_padded)),dtype=torch.int64))
            segment_labels_final.append(torch.tensor(flatten(segment_label_padded),dtype=torch.int64))
            is_nexts_final.append(is_next)

        else:
          # Flatten the padded inputs and labels
            bert_inputs_final.append(flatten(bert_input_padded))
            bert_labels_final.append(flatten(bert_label_padded))
            segment_labels_final.append(flatten(segment_label_padded))
            is_nexts_final.append(is_next)

    return bert_inputs_final, bert_labels_final, segment_labels_final, is_nexts_final

In [18]:
bert_inputs_final, bert_labels_final, segment_labels_final, is_nexts_final=prepare_bert_final_inputs(bert_input, bert_label, is_next,to_tenor=True)
torch.set_printoptions(linewidth=10000)# this assures that whole output is printed in one line
print("input:\t\t",bert_input,"\ninputs_final:\t",bert_inputs_final,"\nbert labels final:\t",bert_labels_final,"\nsegment labels final:\t",segment_labels_final,"\nis nexts final:\t",is_nexts_final)

input:		 [[['[CLS]', 'he', 'likes', 'playing', 'guitar', '[SEP]'], ['i', 'love', 'apples', '[SEP]']]] 
inputs_final:	 [tensor([    1,    33,  1155,   404,  4833,     2,    16,   123, 14227,     2,     0,     0])] 
bert labels final:	 [tensor([ 0, 33,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0])] 
segment labels final:	 [tensor([1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 0, 0])] 
is nexts final:	 [0]


In [19]:
print("input:\t\t",bert_input,"\nmask_label:\t",bert_label, "\nlabels_final: \t",bert_labels_final)

input:		 [[['[CLS]', 'he', 'likes', 'playing', 'guitar', '[SEP]'], ['i', 'love', 'apples', '[SEP]']]] 
mask_label:	 [[['[PAD]', 'he', '[PAD]', '[PAD]', '[PAD]', '[PAD]'], ['[PAD]', '[PAD]', '[PAD]', '[PAD]']]] 
labels_final: 	 [tensor([ 0, 33,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0])]


In [20]:
print("\ninputs_final:\t",bert_inputs_final,"\nsegment_labels:\t",segment_labels_final)


inputs_final:	 [tensor([    1,    33,  1155,   404,  4833,     2,    16,   123, 14227,     2,     0,     0])] 
segment_labels:	 [tensor([1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 0, 0])]


## Preparing the dataset

In [22]:
import csv
from tqdm import tqdm
import json

csv_file_path ='train_bert_data_new.csv'
with open(csv_file_path, mode='w', newline='', encoding='utf-8') as file:
    csv_writer = csv.writer(file)
    csv_writer.writerow(['Original Text', 'BERT Input', 'BERT Label', 'Segment Label', 'Is Next'])

    # Wrap train_iter with tqdm for a progress bar
    for n, (_, sample) in enumerate(tqdm(train_iter, desc="Processing samples")):
        # Tokenize the sample input
        tokens = tokenizer(sample)
        # Create MLM inputs and labels
        bert_input, bert_label = prepare_for_mlm(tokens, include_raw_tokens=False)
        if len(bert_input) < 2:
            continue
        # Create NSP pairs, token labels, and is_next label
        bert_inputs, bert_labels, is_nexts = process_for_nsp(bert_input, bert_label)
        # add zero-paddings, map tokens to vocab indices and create segment labels
        bert_inputs, bert_labels, segment_labels, is_nexts = prepare_bert_final_inputs(bert_inputs, bert_labels, is_nexts)
        # convert tensors to lists, convert lists to JSON-formatted strings
        for bert_input, bert_label, segment_label, is_next in zip(bert_inputs, bert_labels, segment_labels, is_nexts):
            bert_input_str = json.dumps(bert_input.tolist())
            bert_label_str = json.dumps(bert_label.tolist())
            segment_label_str = ','.join(map(str, segment_label.tolist()))
            # Write the data to a CSV file row-by-row
            csv_writer.writerow([sample, bert_input_str, bert_label_str, segment_label_str, is_next])

Processing samples: 900it [06:57,  2.16it/s]


KeyboardInterrupt: 

## Exercises

### Exercise 1 - Initializing the BERTTokenizer

In [23]:
from transformers import BertTokenizer

# Load a pre-trained BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.11/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

### Exercise 2 - Tokenizing the dataset

In [24]:
def yield_tokens(data_iter):
    for _, data_sample in data_iter:
        # Use the BERT tokenizer to tokenize the text
        # This returns a dictionary with 'input_ids' among other things
        tokens = tokenizer(data_sample, return_tensors='pt', truncation=True, max_length=512)['input_ids'][0]
        yield tokens.tolist()

### Exercise 3 - Building the vocabulary with special tokens

In [25]:
from torchtext.data.functional import to_map_style_dataset
from torchtext.datasets import IMDB

# Define special symbols and indices
PAD_IDX, CLS_IDX, SEP_IDX, MASK_IDX, UNK_IDX = tokenizer.pad_token_id, tokenizer.cls_token_id, tokenizer.sep_token_id, tokenizer.mask_token_id, tokenizer.unk_token_id
special_symbols = ['[PAD]', '[CLS]', '[SEP]', '[MASK]', '[UNK]']

# Load IMDB dataset
train_iter, test_iter = IMDB(split=('train', 'test'))

# Convert to map-style datasets to be compatible with transformers' tokenizers
train_dataset = to_map_style_dataset(train_iter)
test_dataset = to_map_style_dataset(test_iter)

# Since you are using a pre-trained tokenizer, you don't need to build the vocab from scratch.
# Instead, you can directly use the tokenizer's vocab.
VOCAB_SIZE = len(tokenizer)

print("Vocabulary Size:", VOCAB_SIZE)

Vocabulary Size: 30522


In [26]:
!pip freeze > requirements.txt